## Installation

In [1]:
pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |████████████████████████████████| 1.3 MB 44.6 MB/s 
     |████████████████████████████████| 4.7 MB 42.3 MB/s 
     |████████████████████████████████| 365 kB 68.5 MB/s 
     |████████████████████████████████| 212 kB 76.2 MB/s 
     |████████████████████████████████| 141 kB 68.0 MB/s 
     |████████████████████████████████| 115 kB 70.9 MB/s 
     |████████████████████████████████| 101 kB 13.7 MB/s 
     |████████████████████████████████| 596 kB 70.7 MB/s 
     |████████████████████████████████| 127 kB 73.7 MB/s 
     |████████████████████████████████| 6.6 MB 10.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:


In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Model 

In [3]:
pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from happytransformer import HappyTextToText

In [5]:
happy_tt = HappyTextToText("T5", "t5-base")

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,
08/10/2022 08:43:26 - INFO - happytransformer.happy_transformer -   Using model: cuda


## Data Collection  

In [6]:
from datasets import load_dataset

In [7]:
train_dataset = load_dataset("jfleg", split='validation[:]')

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

Dataset jfleg downloaded and prepared to /root/.cache/huggingface/datasets/jfleg/default/1.0.0/ed4ab2367351fe31949f48849ae6732b164f0d5ea6bb5d4357ff4293ac89511b. Subsequent calls will reuse this data.


In [8]:
eval_dataset = load_dataset("jfleg", split='test[:]')

## Data Examination  

In [9]:
for case in train_dataset["corrections"][:2]:
  print(case)
  print(case[0])
  print("--------------------------------------------------------")


['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
So I think we would not be alive if our ancestors did not develop sciences and technologies . 
--------------------------------------------------------
['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
Not for use with a car . 
--------------------------------------------------------


## Data Preprocessing   

In [10]:
import csv

In [11]:
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases are blank strings. So we'll skip them
                if input_text and correction:
                    writter.writerow([input_text, correction])
              

In [14]:
generate_csv("jfleg_train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)

## Before Training Evaluating

In [15]:
before_result = happy_tt.eval("eval.csv")

08/10/2022 08:44:31 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-926ec0d2b5ed0106/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


In [16]:
print("Before loss: ", before_result.loss)

Before loss:  1.2803919315338135


## Train

In [17]:
from happytransformer import TTTrainArgs

In [18]:
args = TTTrainArgs(batch_size=8)

In [20]:
happy_tt.train("train.csv", args=args)

08/10/2022 08:48:02 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2aa9dcb2707be928/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

08/10/2022 08:48:06 - INFO - happytransformer.happy_transformer -   Training...
PyTorch: setting up devices
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3016
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1131


Step,Training Loss
500,0.577400
1000,0.441900




Training completed. Do not forget to share your model on huggingface.co/models =)




## After Training Evaluating

In [21]:
after_result = happy_tt.eval("eval.csv")

08/10/2022 08:58:14 - INFO - happytransformer.happy_transformer -   Preprocessing evaluating data...


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 1


In [22]:
print("After loss:", after_result.loss)

After loss: 0.44996318221092224


## Inference

In [23]:
from happytransformer import TTSettings


In [24]:
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

### Example 1

In [25]:
example_1 = "This sentences, has bads grammar and spelling!"

In [26]:
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1.text)

This sentence has bad grammar and spelling!


### Example 2

In [27]:
example_2 = "I am enjoys, writtings articles ons AI."

In [28]:
result_2 = happy_tt.generate_text(example_2, args=beam_settings)
print(result_2.text)

I enjoy writing articles on AI.


In [36]:
example_3 = "I is a student"
res_3 = happy_tt.generate_text(example_3, args=beam_settings)
print(res_3.text)

I am a student.


## Data Preprocessing 

In [29]:
replacements = [
  (" .", "."), 
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')

]

In [30]:
def remove_excess_spaces(text):
  for rep in replacements:
    text = text.replace(rep[0], rep[1])

  return text

In [31]:
remove_excess_spaces("Hi , your seat number is 5 5 .")

'Hi, your seat number is 55.'

In [32]:
def generate_csv_updated(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases are blank strings. So we'll skip them
                if input_text and correction:
                  input_text = remove_excess_spaces(input_text)
                  correction = remove_excess_spaces(correction)
                  writter.writerow([input_text, correction])

In [33]:
 generate_csv_updated("train_2.csv", train_dataset)

## Pretrained Model

In [34]:
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp0tap30t6


storing https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/b8024a4f1b2fa6d7cf28f3c534358586d43cb7595e39b11bf558905f2790ea0c.0433a1687f5de97fb7c003699fe7c8d5449f28334e0509241779b920173f4ad7
creating metadata file for /root/.cache/huggingface/transformers/b8024a4f1b2fa6d7cf28f3c534358586d43cb7595e39b11bf558905f2790ea0c.0433a1687f5de97fb7c003699fe7c8d5449f28334e0509241779b920173f4ad7
loading configuration file https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/b8024a4f1b2fa6d7cf28f3c534358586d43cb7595e39b11bf558905f2790ea0c.0433a1687f5de97fb7c003699fe7c8d5449f28334e0509241779b920173f4ad7
Model config T5Config {
  "_name_or_path": "vennify/t5-base-grammar-correction",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu

storing https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/c7ca604aa817a6ca7f7d6fa4dc0fd73a1ce5cc0d46c55024a4bd440f0a5e8ff1.f53d02421d8b2746cbd27189bbbf75e80717e92ffcc646f7c86092474c31043b
creating metadata file for /root/.cache/huggingface/transformers/c7ca604aa817a6ca7f7d6fa4dc0fd73a1ce5cc0d46c55024a4bd440f0a5e8ff1.f53d02421d8b2746cbd27189bbbf75e80717e92ffcc646f7c86092474c31043b
loading weights file https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/c7ca604aa817a6ca7f7d6fa4dc0fd73a1ce5cc0d46c55024a4bd440f0a5e8ff1.f53d02421d8b2746cbd27189bbbf75e80717e92ffcc646f7c86092474c31043b
All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at vennify/t5-base-grammar-correction.
If your task is simila

storing https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/f71cb5195a493ab4c344b6fb8f158e18384d81d6cd18884221bd00746e53fd46.6f80f46088ffac2a8f61f3cbda0f9d0bd40e0c989dc40ee63d01cb25cc105aa8
creating metadata file for /root/.cache/huggingface/transformers/f71cb5195a493ab4c344b6fb8f158e18384d81d6cd18884221bd00746e53fd46.6f80f46088ffac2a8f61f3cbda0f9d0bd40e0c989dc40ee63d01cb25cc105aa8
https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/spiece.model not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpwbchap3a


storing https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/spiece.model in cache at /root/.cache/huggingface/transformers/e3a77ea58e0241f9d2e819647b964c4a997acd54522c6c12552cd9970e57687a.d6f0605ae3d57070be74b4c12206072ab332922acff822e6b5458691dbda7551
creating metadata file for /root/.cache/huggingface/transformers/e3a77ea58e0241f9d2e819647b964c4a997acd54522c6c12552cd9970e57687a.d6f0605ae3d57070be74b4c12206072ab332922acff822e6b5458691dbda7551
https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp114rxr9n


storing https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/986816064146207c122899bec5d75d10c6e8f3f47e6438056fb448f0c626a700.9d3c2545d152ced836d869f7d99080767f00ce5011f953f2c06baec5e2ee0c40
creating metadata file for /root/.cache/huggingface/transformers/986816064146207c122899bec5d75d10c6e8f3f47e6438056fb448f0c626a700.9d3c2545d152ced836d869f7d99080767f00ce5011f953f2c06baec5e2ee0c40
https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpn9rdhkps


storing https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/611d3f9601141f239482ea797eb43f107d840520c807ac0078e04c5719c0908e.c94798918c92ded6aeef2d2f0e666d2cc4145eca1aa6e1336fde07f2e13e2f46
creating metadata file for /root/.cache/huggingface/transformers/611d3f9601141f239482ea797eb43f107d840520c807ac0078e04c5719c0908e.c94798918c92ded6aeef2d2f0e666d2cc4145eca1aa6e1336fde07f2e13e2f46
loading file https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/spiece.model from cache at /root/.cache/huggingface/transformers/e3a77ea58e0241f9d2e819647b964c4a997acd54522c6c12552cd9970e57687a.d6f0605ae3d57070be74b4c12206072ab332922acff822e6b5458691dbda7551
loading file https://huggingface.co/vennify/t5-base-grammar-correction/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/986816064146207c122899bec5d75d10c6e8f3f47e6438056fb448f0c626a700.9d3c2545d152ced836d869f7d99

In [35]:
result = happy_tt.generate_text("grammar: I boughts ten apple.", args=beam_settings)
print(result.text)

I bought ten apples.
